In [1]:
import GMesh
import math
import netCDF4
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Read topo data
# !cp -n /archive/gold/datasets/topography/GEBCO_08_v2.nc .
with netCDF4.Dataset('GEBCO_08_v2.nc') as nc:
    topo_lon = nc.variables['longitude'][:].filled(0.)
    topo_lat = nc.variables['latitude'][:].filled(0.)
    topo_depth = nc.variables['depth'][:,:].filled(0.)
#! cp -n /archive/gold/datasets/topography/GEBCO_2023/GEBCO_2023.nc .
# with netCDF4.Dataset('/archive/gold/datasets/topography/GEBCO_2023/GEBCO_2023.nc') as nc:
#     topo_lon = nc.variables['lon'][:].filled(0.)
#     topo_lat = nc.variables['lat'][:].filled(0.)
#     topo_depth = nc.variables['elevation'][:,:].filled(0.)

In [3]:
import importlib
importlib.reload(GMesh)

<module 'GMesh' from '/local2/home/GitHub/thin-wall-topography/python/GMesh.py'>

In [4]:
src_topo_global = GMesh.UniformEDS( topo_lon, topo_lat, topo_depth )
src_topo_global

<UniformEDS 43200 x 21600
lon = <RegularCoord n=43200, dx=0.008333333333333333, rdx=120.0, x0=-180.0, io=-21600, rem=0.0, is-ie=0-43200, periodic=True>
[-179.99583333 -179.9875     -179.97916667 ...  179.97916667  179.9875
  179.99583333]
[-180.         -179.99166667 -179.98333333 ...  179.98333333  179.99166667
  180.        ]
lat = <RegularCoord n=21600, dx=0.008333333333333333, rdx=120.0, x0=-90, io=-10800, rem=0.0, is-ie=0-21600, periodic=False>
[-89.99583333 -89.9875     -89.97916667 ...  89.97916667  89.9875
  89.99583333]
[-90.         -89.99166667 -89.98333333 ...  89.98333333  89.99166667
  90.        ]
data = (21600, 43200)>

In [5]:
def convol( levels, h, f, verbose=False ):
    """Coarsens the product of h*f across all levels"""
    levels[-1].height = ( h * f ).reshape(levels[-1].nj,levels[-1].ni)
    for k in range( len(levels) - 1, 0, -1 ):
        if verbose: print('Coarsening {} -> {}'.format(k,k-1))
        levels[k].coarsenby2( levels[k-1] )
    return levels[0].height

In [6]:
def rough( levels, h, h2min=1.e-7 ):
    """Calculates both mean of H, and variance of H relative to a plane"""
    # Construction weights for momenet calculations
    nx = 2**( len(levels) - 1 )
    x = ( np.arange(nx) - ( nx - 1 ) /2 ) * np.sqrt( 12 / ( nx**2 - 1 ) )
    X, Y = np.meshgrid( x, x )
    X, Y = X.reshape(1,nx,1,nx), Y.reshape(1,nx,1,nx)
    h = h.reshape(levels[0].nj,nx,levels[0].ni,nx)
    H2 = convol( levels, h, h )
    HX = convol( levels, h, X )
    HY = convol( levels, h, Y )
    H = convol( levels, h, np.ones((1,nx,1,nx)) )
    return H, H2 - H**2 - HX**2 - HY**2 + h2min

In [7]:
# Read target mesh
with netCDF4.Dataset('/local2/home/GitHub/OM4_025-grid-data/ocean_hgrid.nc') as nc:
    fullG = GMesh.GMesh( lon=nc.variables['x'][::2,::2], lat=nc.variables['y'][::2,::2] )

In [25]:
NtileI, NtileJ = 6, 45
# NtileI, NtileJ = 2, 15
di, dj = fullG.ni // NtileI, fullG.nj // NtileJ
print('window size dj,di =',dj,di,'full model nj,ni=',fullG.nj, fullG.ni)
Hcnt = np.zeros((fullG.nj, fullG.ni)) # Diagnostic: counting which cells we are working on
Htarg, H2targ = np.zeros((fullG.nj, fullG.ni)), np.zeros((fullG.nj, fullG.ni))
gtic = GMesh.GMesh._toc(None,"")
for j in range( NtileJ ): # 13
    csj, sj = slice( j*dj, (j+1)*dj ), slice( j*dj, (j+1)*dj+1 )
    for i in range( NtileI ): # 1
        csi, si = slice( i*di, (i+1)*di ), slice( i*di, (i+1)*di+1 ) # Slices of target grid
        Hcnt[csj,csi] = Hcnt[csj,csi] + 1 # Diagnostic: counting which cells we are working on
        G = GMesh.GMesh( lon=fullG.lon[sj,si], lat=fullG.lat[sj,si] )
        print('J,I={},{} {}%, {}\n   window lon={}:{}, lat={}:{}\n   jslice={}, islice={}'.format( \
            j, i, int(100*(j*NtileI+i)/(NtileI*NtileJ)), G, G.lon.min(), G.lon.max(), G.lat.min(), G.lat.max(), sj, si ))
        # This subsets the source topography so we don't work with too large arrays
        tslices = src_topo_global.bb_slices( G.lon, G.lat )
        print('   topo slices ={}\n   topo lon={}:{}, lat={},{}'.format( \
            tslices, topo_lon[tslices[0]].min(), topo_lon[tslices[0]].max(), topo_lat[tslices[1]].min(), topo_lat[tslices[1]].max() ) )
        # This recursively refines the mesh until some criteria is met ...
        levels = G.refine_loop( topo_lon[tslices[0]], topo_lat[tslices[1]], resolution_limit=False, fixed_refine_level=6 )
        # Use nearest neighbor topography to populate the finest grid
        levels[-1].project_source_data_onto_target_mesh( src_topo_global )
        # Now recursively coarsen
        h, h2 = rough( levels, levels[-1].height )
        # Store window in final array
        Htarg[csj,csi] = h
        H2targ[csj,csi] = h2
GMesh.GMesh._toc(gtic,"Whole workflow")
print( Hcnt.min(), Hcnt.max(), '<-- should both be 1 for full model' )

window size dj,di = 24 240 full model nj,ni= 1080 1440
J,I=0,0 0%, <GMesh nj:24 ni:240 shape:(24,240)>
   window lon=-299.99458164885345:-240.0112484969494, lat=-80.01397197926609:-78.34532374100719
   jslice=slice(0, 25, None), islice=slice(0, 241, None)
   topo slices =(slice(28800, 35999, None), slice(1198, 1399, None))
   topo lon=60.00416666666666:119.98750000000001, lat=-80.0125,-78.34583333333333
Refine level 0 <GMesh nj:24 ni:240 shape:(24,240)> (0.0879 Mb)
Refine level 1 <GMesh nj:48 ni:480 shape:(48,480)> (0.3516 Mb)
Refine level 2 <GMesh nj:96 ni:960 shape:(96,960)> (1.4062 Mb)
Refine level 3 <GMesh nj:192 ni:1920 shape:(192,1920)> (5.6250 Mb)
Refine level 4 <GMesh nj:384 ni:3840 shape:(384,3840)> (22.5000 Mb)
Refine level 5 <GMesh nj:768 ni:7680 shape:(768,7680)> (90.0000 Mb)
Refine level 6 <GMesh nj:1536 ni:15360 shape:(1536,15360)> (360.0000 Mb)
J,I=0,1 0%, <GMesh nj:24 ni:240 shape:(24,240)>
   window lon=-240.34021923567104:-180.02607218723773, lat=-80.91667601006795:-7

In [26]:
with netCDF4.Dataset('new_topo_OM4_grid.nc'.format(NtileI,NtileJ),'w','clobber') as nc:
    nx = nc.createDimension('nx', Htarg.shape[1])
    ny = nc.createDimension('ny', Htarg.shape[0])
    ntiles = nc.createDimension('ntiles', 1)
    z = nc.createVariable('depth', float, ('ny','nx') )
    z2 = nc.createVariable('h2', float, ('ny','nx') )
    z[:,:] = -Htarg[:,:]
    z2[:,:] = H2targ[:,:]